In [4]:
from transformers import pipeline , AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
from utils import tokenize_hfmodel_inputs
device='cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
low_coding_models=['meta-llama/CodeLlama-7b-Instruct-hf','bigcode/starcoder2-7b','microsoft/Phi-3-mini-128k-instruct']
med_coding_models=['codellama/CodeLlama-13b-hf','bigcode/starcoder2-15b']
high_coding_models=['mistralai/Codestral-22B-v0.1','codellama/CodeLlama-34b-Instruct-hf','deepseek-ai/deepseek-coder-33b-instruct']

Text completions

In [6]:
model_name=low_coding_models[0]

In [14]:
sample_text="How can we design linkedlist in python"

In [ ]:
model=AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokens,tokeniser=tokenize_hfmodel_inputs(sample_text,model_name,device)
x=tokens.to(device)

In [9]:
for _ in range(200):
    ops=model(input_ids=x)
    logits=ops.logits
    probs=F.softmax(logits[:,-1,:],dim=-1)

    next_token=torch.multinomial(probs.view(probs.shape[1]),1)

    x=torch.cat([x,next_token.view(1,1)],dim=-1)
op=''
for i in x.tolist():
    op=op+tokeniser.decode(i)
print(op)

<s> def fibanocci(a,b):
    if a == 0 or b == 0:
        return 0
    elif a == 1 or b == 1:
        return 1
    else:
        return fibanocci(a-1) + fibanocci(a-2)
#print(fibanocci(4, 6))

fizz_buzz = lambda x,n: ['fizz' if i%3 ==0 else ' ' and 'buzz' if i%5== 0 else ' ' for i in range(1,x)]
#print(fizz_buzz(16,5))

#from itertools import accumulate
#print(list(accumulate([])))

my_dict = {}
my_dict = dict(sorted(my_dict.items(),key = lambda x: x[1]))

#my_ls = [1,2,5,3,4]
#upper_bound = 100000
#y = range(0, upper_bound, 50000)
#chunks = (


In [15]:
input_ids = tokeniser(sample_text, return_tensors="pt").to(device)
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


How can we design linkedlist in python

How can we design linkedlist in python

Comment: What do you mean by "design"?

Comment: http://www.python-course.eu/linked_lists.php

Answer: \begin{code}
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None

    def append(self, data):
        if self.head is None:
            self.head = Node(data)
            return

        node = self.head
        while node.next is not None:
            node = node.next

        node.next = Node(data)

    def print(self):
        if self.head is None:
            return

       


Chat

In [2]:
model_name="google/gemma-2b-it"

In [ ]:
sample_text="write a c program to print first 5 numbers of fibonacci series."
model=AutoModelForCausalLM.from_pretrained(model_name)
tokeniser=AutoTokenizer.from_pretrained(model)
messages = [{"role": "user", "content": sample_text}]
prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")
output_ids = model.generate(input_ids.input_ids,max_length=200,do_sample=True,temperature=0.7,top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)
print(response)

In [ ]:
messages.append({"role": "assistant", "content": response})
messages.append({"role": "user", "content": "But you did not write c=a+b anywhere"})

prompt = tokeniser.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
input_ids = tokeniser(prompt, return_tensors="pt")

output_ids = model.generate(input_ids.input_ids, max_length=1200, do_sample=True, top_p=0.9)
response = tokeniser.decode(output_ids[0], skip_special_tokens=True)

print(response)
